In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets
from sklearn.ensemble import RandomForestRegressor as rf, GradientBoostingRegressor as gbr, BaggingRegressor as br
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor as knr
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.set_option("display.max.columns", None)
# pd.set_option("display.max.rows", None)

In [2]:
#storing the data from a file path to the varriable
house_price_train_dataset_path = '/kaggle/input/hse-m-psmsimmo-p-347795-1-apartmentprice/train.csv'
house_price_train_dataset_table = pd.read_csv(house_price_train_dataset_path)
house_price_test_dataset_path = '/kaggle/input/hse-m-psmsimmo-p-347795-1-apartmentprice/test.csv'
house_price_test_dataset_table = pd.read_csv(house_price_test_dataset_path)
(house_price_test_dataset_table. index)
house_price_train_dataset_table.isnull().sum()


Id                 0
Room               0
Price              0
Size               0
Balcony        12750
Floor              0
FloorsTotal        0
Walls           5885
Age              541
Lift            9426
District         357
Okrug            357
dtype: int64

In [3]:
# deleting deletable rows with empty cells if they are not much
house_price_train_dataset_table.dropna(subset=['Age','District','Okrug'], inplace=True)

#Replacing  undeletable rows with empty cells with unknown for those cells
house_price_train_dataset_table = house_price_train_dataset_table.fillna("Unknown")
house_price_test_dataset_table = house_price_test_dataset_table.fillna("Unknown")



In [4]:
# appending new row to train dataset that has values in its cells that are present in test data but not in train beofre creating dummies.
new_row_train = {'Id': 1111111111, 'Room': 0, 'Price': 0.0, 'Size': 0, 'Balcony': 'Unknown', 'Floor': 0, 'FloorsTotal': 0, 'Walls': 'Unknown', 'Age': 'Unknown', 'Lift': 'Unknown', 'District': 'Unknown', 'Okrug': 'Unknown'  }
house_price_train_dataset_table = house_price_train_dataset_table.append(new_row_train, ignore_index=True)

# appending new row to test dataset that has values in its cells that are present in train data but not in train beofre creating dummies.
new_row_test = {'Id': 1111111111, 'Room': 0, 'Size': 0, 'Balcony': 'Unknown', 'Floor': 0, 'FloorsTotal': 0, 'Walls': 'Unknown', 'Age': 'Unknown', 'Lift': 'Unknown', 'District': 'Внуково', 'Okrug': 'СВАО'}
house_price_test_dataset_table = house_price_test_dataset_table.append(new_row_test, ignore_index=True)

# creating dummies for test and train data
house_price_train_dataset_table = pd.get_dummies(house_price_train_dataset_table, columns=['Balcony','Walls','Age','Lift','District','Okrug'])
house_price_test_dataset_table = pd.get_dummies(house_price_test_dataset_table, columns=['Balcony','Walls','Age','Lift','District','Okrug'])


#Deleting ID and the rows created to get the dummies before fedding it to the model
house_price_train_dataset_table = house_price_train_dataset_table[house_price_train_dataset_table.Id != 1111111111]
house_price_test_dataset_table = house_price_test_dataset_table[house_price_test_dataset_table.Id != 1111111111]
house_price_train_dataset_table = house_price_train_dataset_table.drop(['Id'], axis=1)
house_price_test_dataset_table =  house_price_test_dataset_table.drop(['Id'], axis=1)

In [5]:
# pd.set_option("display.max.rows", None)
# house_price_test_dataset_table.tail()
# y_train.head()
# house_price_train_dataset_table.isnull().sum()[0:60]
# y_train.shape
# print(house_price_test_dataset_table['Okrug'].unique())
# x_train.dtypes
# print(house_price_test_dataset_table.loc[~house_price_test_dataset_table['District'].isin(house_price_train_dataset_table['District']), 'District'])
# print(house_price_test_dataset_table[house_price_test_dataset_table['District'] == 'Внуково'])
# print(house_price_test_dataset_table.columns.difference(house_price_train_dataset_table.columns))

# corr = house_price_train_dataset_table.corr()
# print (corr['Price'].sort_values(ascending=False)[120:180])

#undferstanding the correlation between the various features in the data(positive or negative)
# correlation=house_price_train_dataset_table.corr()
# plt.figure(figsize=(20,20))
# sns.heatmap(correlation, cbar=True, square=True, fmt='.1f', annot=True, annot_kws={'size':5}, cmap='Blues')

# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=2)
# print(X.shape, X_train.shape, X_test.shape )

# checking errors with R Sqaured and mean absolute error
# score_1=metrics.r2_score(Y_train, training_data_prediction)
# score_2=metrics.mean_absolute_error(Y_train, training_data_prediction)
# print('R square error: ', score_1)
# print('mean absolute error: ', score_2)

In [6]:
# spliting the data
x_train=house_price_train_dataset_table.drop(['Price'], axis=1)
y_train=house_price_train_dataset_table['Price']
x_test=house_price_test_dataset_table

In [7]:
model=XGBRegressor()
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [8]:
# Reorder the test dataframe to match the order of the train dataframe
x_test = x_test.reindex(columns = x_train.columns)

# predicting the data
test_data_prediction=model.predict(x_test)
test_data_prediction=pd.DataFrame(test_data_prediction)
pd.options.display.float_format = '{:.8f}'.format
test_data_prediction.index.name = 'Id'
test_data_prediction.index = test_data_prediction.index + 40509
test_data_prediction.rename(columns = {0:'Price'}, inplace = True)
test_data_prediction.to_csv(f"Submission_file_oct-2021-101.csv")
# res.to_csv("prediction_results.csv")
print(test_data_prediction)




                   Price
Id                      
40509   9665205.00000000
40510 172610496.00000000
40511 115669616.00000000
40512 106420440.00000000
40513  29160420.00000000
...                  ...
50631  12626517.00000000
50632  11553871.00000000
50633   5278871.00000000
50634  10600226.00000000
50635   9266459.00000000

[10127 rows x 1 columns]
